In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nglview as nv

In [8]:
import molsysmt as msm

AttributeError: 'super' object has no attribute '_ipython_display_'

In [ ]:
pdbs_to_be_tested = ["1BG3", "2PGI", "4PFK", "4ALD", "3YPI", "3GPD",
                    "13PK", "1BQ3", "7ENL", "1A3W"]

In [ ]:
molsys = msm.convert('1BG3')

In [ ]:
molsys.topology.atoms_dataframe['b_factor'][[0,1,2,3]].to_numpy()

In [2]:
import pandas as pd
import pint
import pint_pandas

In [3]:
df = pd.DataFrame(columns=['length', 'volume'])

In [25]:
df['volume']=pint_pandas.PintArray([0,1,2,3], dtype='dm**3')
df['length']=pint_pandas.PintArray([0,1,2,3], dtype='m')

In [26]:
df.dtypes

length             pint[meter]
volume    pint[decimeter ** 3]
dtype: object

In [28]:
df.iloc[1]['volume']

1.0 <Unit('decimeter ** 3')>